In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
from os.path  import join
import struct
from array import array

In [4]:
class MyPCA:
    def __init__(self, n_components):
        self.n_components = n_components   
        self.components = None
        self.mean = None
        self.scale = None
        
    def fit(self, X):
        # Standardize data 
        X = X.copy()
        self.mean = np.mean(X, axis = 0)
        self.scale = np.std(X, axis = 0)
        
        # --- FIX LỖI NaN/Inf ---
        # Nếu độ lệch chuẩn bằng 0 (pixel không đổi), gán bằng 1 để tránh chia cho 0
        self.scale[self.scale == 0] = 1.0
        # -----------------------
        
        X_std = (X - self.mean) / self.scale
        
        # Eigendecomposition of covariance matrix       
        # Dùng rowvar=False vì mỗi cột là 1 feature
        cov_mat = np.cov(X_std, rowvar=False)
        
        # Dùng eigh cho ma trận đối xứng (nhanh và ổn định hơn eig)
        eig_vals, eig_vecs = np.linalg.eigh(cov_mat) 
        
        # Sắp xếp giảm dần (eigh trả về tăng dần nên cần đảo ngược)
        sorted_indices = np.argsort(eig_vals)[::-1]
        eig_vals_sorted = eig_vals[sorted_indices]
        eig_vecs_sorted = eig_vecs[:, sorted_indices]
        
        # Chỉnh dấu (Sign flip) để nhất quán
        max_abs_idx = np.argmax(np.abs(eig_vecs_sorted), axis=0)
        signs = np.sign(eig_vecs_sorted[max_abs_idx, range(eig_vecs_sorted.shape[1])])
        eig_vecs_sorted = eig_vecs_sorted * signs[np.newaxis, :]
        
        # Lưu các thành phần chính (Transpose để tiện dot product sau này: (n_components, n_features))
        self.components = eig_vecs_sorted[:, :self.n_components].T
        
        # Explained variance ratio
        total_variance = np.sum(eig_vals_sorted)
        self.explained_variance_ratio = eig_vals_sorted[:self.n_components] / total_variance
        self.cum_explained_variance = np.cumsum(self.explained_variance_ratio)

        return self

    def transform(self, X):
        X = X.copy()
        # Dùng scale đã xử lý số 0 từ bước fit
        X_std = (X - self.mean) / self.scale
        # Chiếu dữ liệu: X_std (N, 784) dot Components.T (784, k) -> (N, k)
        # Lưu ý: self.components ở trên mình đã Transpose dạng (k, 784)
        # Nên ở đây nhân với self.components.T (784, k)
        X_proj = X_std.dot(self.components.T)
        
        return X_proj

In [ ]:
import numpy as np
import struct
from array import array
import os

# ==========================================
# 1. DATA LOADER CLASS (Code của bạn)
# ==========================================
class MnistDataloader(object):
    def __init__(self, training_images_filepath, training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())        
        
        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())        
        
        images = []
        # Cách xử lý mảng này sẽ trả về shape (N, 28, 28)
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img            
        
        return np.array(images), np.array(labels)
            
    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)

# ==========================================
# 2. CÁC HÀM FEATURE ENGINEERING (Đã thiết kế trước đó)
# ==========================================

# --- Design 1: Raw Pixels (Code của bạn) ---
def get_design1_raw_pixels(x_input, mask=None):
    """
    Design 1: Raw Pixels với Variance Thresholding.
    Nếu mask=None, hàm sẽ tính mask (dùng cho tập TRAIN).
    Nếu mask có giá trị, hàm sẽ áp dụng mask đó (dùng cho tập TEST).
    """
 
    # Bước 1: Tính mask nếu chưa có (chỉ tập TRAIN)
    if mask is None:
        feature_mask = np.var(x_input, axis=0) > 0
        n_removed = x_input.shape[1] - np.sum(feature_mask)
        print(f"-> Đã loại bỏ {n_removed} pixel tĩnh (constant pixels).")
    else:
        feature_mask = mask # Sử dụng mask đã truyền vào (tập TEST)
    
    # Bước 2: Lọc bỏ các cột chết
    x_filtered = x_input[:, feature_mask]
    
    # Bước 3: Chuẩn hóa về [0, 1]
    x_norm = x_filtered / 255.0
    
    # Bước 4: Thêm Bias (cột toàn số 1 vào đầu)
    m = x_norm.shape[0]
    x_final = np.hstack([np.ones((m, 1)), x_norm])
    
    # Bước 5: TRẢ VỀ 2 GIÁ TRỊ (vector features và mask)
    return x_final, feature_mask

# --- Design 2: Block Averaging ---
def get_design2_block_avg(x_input, block_size=2):
    # x_input: (N, 784) -> Cần reshape để xử lý
    N = x_input.shape[0]
    images = x_input.reshape(N, 28, 28)
    
    grid_h, grid_w = 28 // block_size, 28 // block_size
    reshaped = images.reshape(N, grid_h, block_size, grid_w, block_size)
    averaged = reshaped.mean(axis=(2, 4)).reshape(N, -1)
    
    x_norm = averaged / 255.0
    return np.hstack([np.ones((N, 1)), x_norm])

# --- Design 3: Projection Profiles ---
def get_design3_projection(x_input, target_dim=100):
    N, D = x_input.shape
    
    np.random.seed(42) 
    
    projection_matrix = np.random.randn(D, target_dim)
  
    x_projected = np.dot(x_input, projection_matrix)
    
    max_val = np.max(np.abs(x_projected))
    x_norm = x_projected / max_val

    return np.hstack([np.ones((N, 1)), x_norm])



def get_design4_pca(x_train_flat, x_test_flat, n_components=100):
    # 1. Fit PCA chỉ trên tập Train
    pca = MyPCA(n_components=n_components).fit(x_train_flat)
    
    # 2. Transform (chiếu dữ liệu)
    # Áp dụng Mean/Scale/Components đã học từ tập train cho cả 2 tập
    x_train_pca = pca.transform(x_train_flat)
    x_test_pca = pca.transform(x_test_flat)
    
    # 3. Báo cáo tỷ lệ phương sai giữ lại (Đưa vào Báo cáo)
    variance_retained = np.sum(pca.explained_variance_ratio) * 100
    # print(f"-> PCA (k={n_components}) giữ lại {variance_retained:.2f}% phương sai.")
    
    # 4. Thêm Bias (cột toàn số 1)
    m_train = x_train_pca.shape[0]
    m_test = x_test_pca.shape[0]
    
    x_train_final = np.hstack([np.ones((m_train, 1)), x_train_pca])
    x_test_final = np.hstack([np.ones((m_test, 1)), x_test_pca])
    
    return x_train_final, x_test_final
 
# --- Helper: One-Hot Encoding ---
def one_hot_encode(y, num_classes=10):
    return np.eye(num_classes)[y]

# ==========================================
# 3. LUỒNG THỰC THI CHÍNH (MAIN PIPELINE)
# ==========================================

# A. Định nghĩa đường dẫn file (Thay đổi đường dẫn thực tế của bạn ở đây)
input_path = './data' # Ví dụ thư mục chứa file
training_images_filepath = os.path.join(input_path, 'train-images.idx3-ubyte')
training_labels_filepath = os.path.join(input_path, 'train-labels.idx1-ubyte')
test_images_filepath = os.path.join(input_path, 't10k-images.idx3-ubyte')
test_labels_filepath = os.path.join(input_path, 't10k-labels.idx1-ubyte')

# B. Load dữ liệu thô
print("1. Loading Data...")
mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(x_train_raw, y_train_raw), (x_test_raw, y_test_raw) = mnist_dataloader.load_data()

# C. Tiền xử lý chung (Global Preprocessing)
print(f"   Original Shape: {x_train_raw.shape}") # Thường là (60000, 28, 28)

# Bắt buộc: Flatten về (N, 784) để thống nhất đầu vào cho các hàm Feature Design
x_train_flat = x_train_raw.reshape(x_train_raw.shape[0], -1)
x_test_flat = x_test_raw.reshape(x_test_raw.shape[0], -1)

# Bắt buộc: One-Hot Encode cho nhãn y
y_train_encoded = one_hot_encode(y_train_raw)
y_test_encoded = one_hot_encode(y_test_raw)

print(f"   Flattened Shape: {x_train_flat.shape}")
print(f"   Y Encoded Shape: {y_train_encoded.shape}")

# D. Tạo 3 bộ dữ liệu Feature Design
print("\n2. Generating Feature Vectors...")

# --- Design 1 ---
# Lưu ý: Tính mask trên Train, áp dụng mask đó cho Test
x_train_d1, mask_d1 = get_design1_raw_pixels(x_train_flat)
x_test_d1, _ = get_design1_raw_pixels(x_test_flat, mask=mask_d1)
print(f"   Design 1 (Raw Pixels) Shape: {x_train_d1.shape}")

# --- Design 2 ---
x_train_d2 = get_design2_block_avg(x_train_flat)
x_test_d2 = get_design2_block_avg(x_test_flat)
print(f"   Design 2 (Block Avg 2) Shape:  {x_train_d2.shape}")

# --- Design 3 ---
x_train_d3 = get_design2_block_avg(x_train_flat, block_size=4)
x_test_d3 = get_design2_block_avg(x_test_flat, block_size=4)
print(f"   Design 3 (Block Avg 4) Shape:  {x_train_d3.shape}")

# --- Design 4 ---
x_train_d4 = get_design3_projection(x_train_flat)
x_test_d4 = get_design3_projection(x_test_flat)
print(f"   Design 4 (Projections) Shape: {x_train_d4.shape}")

# --- Design 5 ---
x_train_d5, x_test_d5 = get_design4_pca(x_train_flat, x_test_flat, n_components=331) 
print(f"   Design 5 (PCA) Shape:      {x_train_d5.shape}")





print("\n-> Sẵn sàng để đưa vào huấn luyện!")

1. Loading Data...
   Original Shape: (60000, 28, 28)
   Flattened Shape: (60000, 784)
   Y Encoded Shape: (60000, 10)

2. Generating Feature Vectors...
-> Đã loại bỏ 67 pixel tĩnh (constant pixels).
   Design 1 (Raw Pixels) Shape: (60000, 718)
   Design 2 (Block Avg 2) Shape:  (60000, 197)
   Design 3 (Block Avg 4) Shape:  (60000, 50)
[[ 0.49671415 -0.1382643   0.64768854 ...  0.26105527  0.00511346
  -0.23458713]
 [-1.41537074 -0.42064532 -0.34271452 ...  0.15372511  0.05820872
  -1.1429703 ]
 [ 0.35778736  0.56078453  1.08305124 ...  0.30729952  0.81286212
   0.62962884]
 ...
 [ 1.0694361  -0.04001221 -1.00736785 ... -0.68887077  0.45739704
   1.39050338]
 [ 0.61764047 -0.86375955  0.61416404 ...  0.81223222 -0.63712757
   0.83060074]
 [ 1.28311075  0.30480564  0.43324489 ...  0.40608518 -0.14288634
   1.94106096]]
[[ 0.49671415 -0.1382643   0.64768854 ...  0.26105527  0.00511346
  -0.23458713]
 [-1.41537074 -0.42064532 -0.34271452 ...  0.15372511  0.05820872
  -1.1429703 ]
 [ 0.357

In [6]:

# ======================================================
# TEST THỬ BẰNG THƯ VIỆN SKLEARN - KHÔNG DÙNG CHÍNH THỨC
# ======================================================
params = {
    'lr': 0.2,       
    'epochs': 1000    
}

results = {}

# Danh sách các bộ dữ liệu đã chuẩn bị
datasets = [
    ("Design 1: Raw Pixels", x_train_d1, x_test_d1),
    ("Design 2: Block Avg-2",  x_train_d2, x_test_d2),
    ("Design 3: Block Avg-4",  x_train_d3, x_test_d3),
    ("Design 4: Projections", x_train_d4, x_test_d4),
    ("Desing 5: PCA", x_train_d5, x_test_d5)
]


y_test_labels = np.argmax(y_test_encoded, axis=1)
